<h1>Datascience Capstone Project</h1>

In this capstone project, hopefully I will learn new skill related location data and how to use it develop cool applciations.

<h1>Assignment - Part 1</h1>

In [1]:
#import numpy and panda
import numpy as np 
import pandas as pd

In [2]:
#import requests for getting the HTML contents of the website and lxml.html for parsing the relevant field
# references : https://towardsdatascience.com/web-scraping-html-tables-with-python-c9baba21059
import requests
import lxml.html as lh

In [3]:
#Create a handle, page, to handle the contents of the website
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

#import beautiful soup
from bs4 import BeautifulSoup

#Store the contents of the website under doc
res = requests.get(url)

In [4]:
#break the cell content into PostalCode, Borough, and Neighborhood
def split_text(txt):
    #print(txt)
    txt = txt.replace('\n','')  #remove all \n
    x = txt[0:3]       # the the postal code
    y = txt[3:len(txt)] 
    z = y.split('(') # split borough and neighborhood
    z.insert(0,x)  # put all into array
    z=np.array(z)
    return(z)


In [23]:
# define the dataframe columns
column_names = ['Postal Code', 'Borough', 'Neighborhood'] 
# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

neighborhoods

,Postal Code,Borough,Neighborhood


In [24]:
soup = BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0] 
    
#iteration through the cell
intial=0;

for row in table.find_all('tr'):
    for cell in row.find_all('td'):
       s_text = split_text(cell.text)
       #print(s_text)
       #print(len(s_text))
       if s_text[1] != "Not assigned":
          postalcode = s_text[0]
          borough = s_text[1]
          if (len(s_text)==2):
             neighborhood_name = s_text[1]   
             neighborhoods = neighborhoods.append({'Postal Code': postalcode, 
                                                   'Borough': borough,
                                                   'Neighborhood': neighborhood_name}, ignore_index=True)
          elif (len(s_text)==3):
             neighborhood_name = s_text[2]
             neighborhood_name = neighborhood_name[0:(len(neighborhood_name) - 1)]      #remove ')'        
             neighborhoods = neighborhoods.append({'Postal Code': postalcode, 
                                                   'Borough': borough, 
                                                   'Neighborhood': neighborhood_name}, ignore_index=True)   
          elif (len(s_text) > 3):
             neighborhood_name = s_text[2]
             neighborhood_name = neighborhood_name[0:(len(neighborhood_name) - 1)]      #remove ')'        
             neighborhoods = neighborhoods.append({'Postal Code': postalcode, 
                                                   'Borough': borough, 
                                                   'Neighborhood': neighborhood_name}, ignore_index=True) 
            

In [25]:
neighborhoods.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Queen's Park / Ontario Provincial Government,Queen's Park / Ontario Provincial Government


In [26]:
neighborhoods.shape

(103, 3)

<h1>Assignment - Part 2</h1>

In [27]:
#Read the coordinate - as geocoder - cannot be imported
geo_loc = pd.read_csv("http://cocl.us/Geospatial_data") 


In [28]:
# Preview the first 5 lines of the loaded data 
geo_loc.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [29]:
geo_loc.shape

(103, 3)

In [34]:
result = neighborhoods.sort_values(['Postal Code'], ascending=1, axis=0)

In [35]:
result.head()

,Postal Code,Borough,Neighborhood
6,M1B,Scarborough,Malvern / Rouge
12,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
18,M1E,Scarborough,Guildwood / Morningside / West Hill
22,M1G,Scarborough,Woburn
26,M1H,Scarborough,Cedarbrae


In [36]:
result.shape

(103, 3)

In [37]:
sorted_neigh = result.reset_index(drop=True) #drop parameter to avoid the old index being added as a column
sorted_neigh.head()

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [42]:
#merge neighboorhood and geospatial data
df_neigh_loc = pd.merge(sorted_neigh, geo_loc, on='Postal Code')

In [43]:
df_neigh_loc.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497
2,M1E,Scarborough,Guildwood / Morningside / West Hill,43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [44]:
df_neigh_loc.shape

(103, 5)

<h1>Assignment - Part 3 <h1>

<h3>Render Toronto city <h3>

In [102]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

Solving environment: done

# All requested packages already installed.



In [53]:
#Toronto Lat & Long
latitude = 43.6532
longitude = -79.3832

In [56]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_neigh_loc['Latitude'], df_neigh_loc['Longitude'], df_neigh_loc['Borough'], df_neigh_loc['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

let's simplify the above map and segment and cluster only the neighborhoods in Scarborough. So let's slice the original dataframe and create a new dataframe of the Scarborough data

In [64]:
scarborough_data = df_neigh_loc[df_neigh_loc['Borough'] == 'Scarborough'].reset_index(drop=True)
scarborough_data.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497
2,M1E,Scarborough,Guildwood / Morningside / West Hill,43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [65]:
scarborough_data.shape

(17, 5)

In [59]:
from geopy.geocoders import Nominatim 

In [77]:
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

In [62]:
address = 'Scarborough, TO'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.773077, -79.257774.


In [66]:
# create map of Manhattan using latitude and longitude values
map_scarborough = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(scarborough_data['Latitude'], scarborough_data['Longitude'], scarborough_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_scarborough)  
    
map_scarborough

Define Foursquare Credentials and Version

In [67]:
CLIENT_ID = 'QJAF2DMIN03XY20HHQMUI0DZ51S2ROBZTBC42ZN2IXD2LLGK' # your Foursquare ID
CLIENT_SECRET = 'U45IJQUY5YAKODVEQ2SOCVRQGMXLWOEU20GCCAXD3EK2QTQ2' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: QJAF2DMIN03XY20HHQMUI0DZ51S2ROBZTBC42ZN2IXD2LLGK
CLIENT_SECRET:U45IJQUY5YAKODVEQ2SOCVRQGMXLWOEU20GCCAXD3EK2QTQ2


In [71]:
scarborough_data.loc[3, 'Neighborhood']

'Woburn'

Get the neighborhood's latitude and longitude values.

In [72]:
neighborhood_latitude = scarborough_data.loc[3, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = scarborough_data.loc[3, 'Longitude'] # neighborhood longitude value

neighborhood_name = scarborough_data.loc[3, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Woburn are 43.7709921, -79.21691740000001.


Create the GET request URL. Name your URL url.

In [73]:
# type your answer here
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=QJAF2DMIN03XY20HHQMUI0DZ51S2ROBZTBC42ZN2IXD2LLGK&client_secret=U45IJQUY5YAKODVEQ2SOCVRQGMXLWOEU20GCCAXD3EK2QTQ2&v=20180605&ll=43.7709921,-79.21691740000001&radius=500&limit=100'

In [75]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [78]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Starbucks,Coffee Shop,43.770037,-79.221156
1,Tim Hortons,Coffee Shop,43.770827,-79.223078
2,Korean Grill House,Korean Restaurant,43.770812,-79.214502
3,Toronto Ali Sami Yen Spor Kompleksi (Lionhill),Soccer Field,43.767670,-79.218210


In [79]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

4 venues were returned by Foursquare.


In [80]:
#function to repeat the same process to all the neighborhoods
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

create a new dataframe called scarborough_venues for each neighborhood

In [81]:
scarborough_venues = getNearbyVenues(names=scarborough_data['Neighborhood'],
                                   latitudes=scarborough_data['Latitude'],
                                   longitudes=scarborough_data['Longitude']
                                  )

Malvern / Rouge
Rouge Hill / Port Union / Highland Creek
Guildwood / Morningside / West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park / Ionview / East Birchmount Park
Golden Mile / Clairlea / Oakridge
Cliffside / Cliffcrest / Scarborough Village West
Birch Cliff / Cliffside West
Dorset Park / Wexford Heights / Scarborough Town Centre
Wexford / Maryvale
Agincourt)
Clarks Corners / Tam O'Shanter / Sullivan
Milliken / Agincourt North / Steeles East / L'Amoreaux East
Steeles West / L'Amoreaux West
Upper Rouge


In [82]:
print(scarborough_venues.shape)
scarborough_venues.head()

(93, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Malvern / Rouge,43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,Guildwood / Morningside / West Hill,43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
3,Guildwood / Morningside / West Hill,43.763573,-79.188711,Marina Spa,43.766000,-79.191000,Spa
4,Guildwood / Morningside / West Hill,43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant


Let's check how many venues were returned for each neighborhood

In [83]:
scarborough_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt),5,5,5,5,5,5
Birch Cliff / Cliffside West,4,4,4,4,4,4
Cedarbrae,8,8,8,8,8,8
Clarks Corners / Tam O'Shanter / Sullivan,14,14,14,14,14,14
Cliffside / Cliffcrest / Scarborough Village West,4,4,4,4,4,4
Dorset Park / Wexford Heights / Scarborough Town Centre,5,5,5,5,5,5
Golden Mile / Clairlea / Oakridge,10,10,10,10,10,10
Guildwood / Morningside / West Hill,7,7,7,7,7,7
Kennedy Park / Ionview / East Birchmount Park,6,6,6,6,6,6


Let's find out how many unique categories can be curated from all the returned venues

In [84]:
print('There are {} uniques categories.'.format(len(scarborough_venues['Venue Category'].unique())))

There are 55 uniques categories.


<h3>Analyze each neighborrhoods<h3>

In [86]:
# one hot encoding
scarborough_onehot = pd.get_dummies(scarborough_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
scarborough_onehot['Neighborhood'] = scarborough_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [scarborough_onehot.columns[-1]] + list(scarborough_onehot.columns[:-1])
scarborough_onehot = scarborough_onehot[fixed_columns]

scarborough_onehot.head()

,Neighborhood,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,...,Playground,Rental Car Location,Sandwich Place,Shopping Mall,Skating Rink,Smoke Shop,Soccer Field,Spa,Thai Restaurant,Vietnamese Restaurant
0,Malvern / Rouge,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Rouge Hill / Port Union / Highland Creek,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Guildwood / Morningside / West Hill,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Guildwood / Morningside / West Hill,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,Guildwood / Morningside / West Hill,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [87]:
scarborough_onehot.shape

(93, 56)

In [89]:
scarborough_grouped = scarborough_onehot.groupby('Neighborhood').mean().reset_index()
scarborough_grouped

,Neighborhood,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,...,Playground,Rental Car Location,Sandwich Place,Shopping Mall,Skating Rink,Smoke Shop,Soccer Field,Spa,Thai Restaurant,Vietnamese Restaurant
0,Agincourt),0.00,0.000,0.000000,0.000000,0.000000,0.0,0.200000,0.0,0.0,...,0.00,0.000000,0.000000,0.000000,0.20,0.000000,0.00,0.000000,0.000000,0.0
1,Birch Cliff / Cliffside West,0.00,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.00,0.000000,0.000000,0.000000,0.25,0.000000,0.00,0.000000,0.000000,0.0
2,Cedarbrae,0.00,0.125,0.000000,0.125000,0.125000,0.0,0.000000,0.0,0.0,...,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.125000,0.0
3,Clarks Corners / Tam O'Shanter / Sullivan,0.00,0.000,0.000000,0.000000,0.071429,0.0,0.000000,0.0,0.0,...,0.00,0.000000,0.000000,0.071429,0.00,0.000000,0.00,0.000000,0.071429,0.0
4,Cliffside / Cliffcrest / Scarborough Village West,0.25,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.00,0.000000,0.000000,0.000000,0.25,0.000000,0.00,0.000000,0.000000,0.0
5,Dorset Park / Wexford Heights / Scarborough To...,0.00,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.2
6,Golden Mile / Clairlea / Oakridge,0.00,0.000,0.000000,0.200000,0.000000,0.0,0.000000,0.2,0.1,...,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.10,0.000000,0.000000,0.0
7,Guildwood / Morningside / West Hill,0.00,0.000,0.000000,0.000000,0.142857,0.0,0.000000,0.0,0.0,...,0.00,0.142857,0.000000,0.000000,0.00,0.000000,0.00,0.142857,0.000000,0.0
8,Kennedy Park / Ionview / East Birchmount Park,0.00,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.0
9,Malvern / Rouge,0.00,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.0


In [91]:
scarborough_grouped.shape

(16, 56)

Print each neighborhood along with the top 5 most common venues

In [92]:
num_top_venues = 5

for hood in scarborough_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = scarborough_grouped[scarborough_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt)----
                       venue  freq
0  Latin American Restaurant   0.2
1                     Lounge   0.2
2             Breakfast Spot   0.2
3               Skating Rink   0.2
4             Clothing Store   0.2


----Birch Cliff / Cliffside West----
                   venue  freq
0        College Stadium  0.25
1           Skating Rink  0.25
2  General Entertainment  0.25
3                   Café  0.25
4                   Park  0.00


----Cedarbrae----
                 venue  freq
0      Thai Restaurant  0.12
1               Bakery  0.12
2                 Bank  0.12
3  Fried Chicken Joint  0.12
4   Athletics & Sports  0.12


----Clarks Corners / Tam O'Shanter / Sullivan----
                  venue  freq
0           Pizza Place  0.14
1    Italian Restaurant  0.07
2  Fast Food Restaurant  0.07
3   Fried Chicken Joint  0.07
4     Convenience Store  0.07


----Cliffside / Cliffcrest / Scarborough Village West----
                 venue  freq
0  American Restaurant  0.25
1 

In [93]:
# sort the venues in descending order.
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Create the new dataframe and display the top 10 venues for each neighborhood.

In [94]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = scarborough_grouped['Neighborhood']

for ind in np.arange(scarborough_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(scarborough_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt),Clothing Store,Skating Rink,Breakfast Spot,Latin American Restaurant,Lounge,College Stadium,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant
1,Birch Cliff / Cliffside West,General Entertainment,Skating Rink,Café,College Stadium,Vietnamese Restaurant,Coffee Shop,Grocery Store,Gas Station,Fried Chicken Joint,Fast Food Restaurant
2,Cedarbrae,Thai Restaurant,Athletics & Sports,Hakka Restaurant,Bakery,Bank,Gas Station,Fried Chicken Joint,Caribbean Restaurant,Construction & Landscaping,Grocery Store
3,Clarks Corners / Tam O'Shanter / Sullivan,Pizza Place,Gas Station,Intersection,Chinese Restaurant,Convenience Store,Noodle House,Pharmacy,Italian Restaurant,Shopping Mall,Bank
4,Cliffside / Cliffcrest / Scarborough Village West,American Restaurant,Skating Rink,Motel,Movie Theater,Coffee Shop,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store


<h3>Cluster Neighborhoods<h3>

In [95]:
# set number of clusters
kclusters = 5

scarborough_grouped_clustering = scarborough_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(scarborough_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 1, 0, 1, 0, 3], dtype=int32)

create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood

In [ ]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

scarborough_merged = scarborough_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
scarborough_merged = scarborough_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

In [98]:
scarborough_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353,3.0,Fast Food Restaurant,Vietnamese Restaurant,Coffee Shop,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Electronics Store,Discount Store,Department Store
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497,2.0,Bar,Vietnamese Restaurant,Coffee Shop,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
2,M1E,Scarborough,Guildwood / Morningside / West Hill,43.763573,-79.188711,1.0,Spa,Bank,Intersection,Rental Car Location,Electronics Store,Medical Center,Mexican Restaurant,Vietnamese Restaurant,College Stadium,Gas Station
3,M1G,Scarborough,Woburn,43.770992,-79.216917,0.0,Coffee Shop,Soccer Field,Korean Restaurant,Vietnamese Restaurant,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,1.0,Thai Restaurant,Athletics & Sports,Hakka Restaurant,Bakery,Bank,Gas Station,Fried Chicken Joint,Caribbean Restaurant,Construction & Landscaping,Grocery Store


In [125]:
import math

 let's visualize the resulting clusters

In [128]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]  

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(scarborough_merged['Latitude'], scarborough_merged['Longitude'], scarborough_merged['Neighborhood'], scarborough_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    if not(math.isnan(cluster)):
        folium.CircleMarker(
           [lat, lon],
           radius=5,
           popup=label,
           color=rainbow[int(cluster-1)],
           fill=True,
           fill_color=rainbow[int(cluster-1)],
           fill_opacity=0.7).add_to(map_clusters)      
map_clusters

<h3> Examine the clusters<h3>

Cluster 1

In [105]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 0, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Scarborough,0.0,Coffee Shop,Soccer Field,Korean Restaurant,Vietnamese Restaurant,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
6,Scarborough,0.0,Discount Store,Coffee Shop,Department Store,Convenience Store,Hobby Shop,Vietnamese Restaurant,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint
7,Scarborough,0.0,Bakery,Bus Line,Ice Cream Shop,Metro Station,Soccer Field,Intersection,Park,Bus Station,Discount Store,Convenience Store
14,Scarborough,0.0,Coffee Shop,Bakery,Playground,Park,Vietnamese Restaurant,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store


cluster 2

In [107]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 1, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Scarborough,1.0,Spa,Bank,Intersection,Rental Car Location,Electronics Store,Medical Center,Mexican Restaurant,Vietnamese Restaurant,College Stadium,Gas Station
4,Scarborough,1.0,Thai Restaurant,Athletics & Sports,Hakka Restaurant,Bakery,Bank,Gas Station,Fried Chicken Joint,Caribbean Restaurant,Construction & Landscaping,Grocery Store
8,Scarborough,1.0,American Restaurant,Skating Rink,Motel,Movie Theater,Coffee Shop,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
9,Scarborough,1.0,General Entertainment,Skating Rink,Café,College Stadium,Vietnamese Restaurant,Coffee Shop,Grocery Store,Gas Station,Fried Chicken Joint,Fast Food Restaurant
10,Scarborough,1.0,Indian Restaurant,Vietnamese Restaurant,Pet Store,Chinese Restaurant,Coffee Shop,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
11,Scarborough,1.0,Auto Garage,Smoke Shop,Bakery,Shopping Mall,Sandwich Place,Breakfast Spot,Vietnamese Restaurant,Electronics Store,Convenience Store,Department Store
12,Scarborough,1.0,Clothing Store,Skating Rink,Breakfast Spot,Latin American Restaurant,Lounge,College Stadium,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant
13,Scarborough,1.0,Pizza Place,Gas Station,Intersection,Chinese Restaurant,Convenience Store,Noodle House,Pharmacy,Italian Restaurant,Shopping Mall,Bank
15,Scarborough,1.0,Chinese Restaurant,Fast Food Restaurant,Coffee Shop,Bank,Nail Salon,Sandwich Place,Breakfast Spot,Grocery Store,Pizza Place,Pharmacy


cluster 3

In [109]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 2, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Scarborough,2.0,Bar,Vietnamese Restaurant,Coffee Shop,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store


cluster 4

In [112]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 3, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,3.0,Fast Food Restaurant,Vietnamese Restaurant,Coffee Shop,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Electronics Store,Discount Store,Department Store


cluster 5

In [111]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 4, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Scarborough,4.0,Playground,Construction & Landscaping,Vietnamese Restaurant,Coffee Shop,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
